In [1]:
import os
print(os.listdir("../input"))
from PIL import Image
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from sklearn import preprocessing
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms, models
import time
import math

['dogs-vs-cats-redux-kernels-edition']


In [2]:
# Train_dir, Test_dir
base_dir = '../input/dogs-vs-cats-redux-kernels-edition'
train_dir = '../data/train'
test_dir = '../data/test'

In [3]:
import zipfile
# Extract All Data From Zip to "../data" Directory
with zipfile.ZipFile(os.path.join(base_dir, 'train.zip')) as train_zip:
    train_zip.extractall('../data')
    
with zipfile.ZipFile(os.path.join(base_dir, 'test.zip')) as test_zip:
    test_zip.extractall('../data')

In [4]:
# Check File Name
os.listdir(train_dir)[:5]

['dog.890.jpg', 'dog.1178.jpg', 'dog.7845.jpg', 'dog.4632.jpg', 'cat.3660.jpg']

In [5]:
img_files = os.listdir(train_dir)

In [6]:
len(img_files)

25000

In [7]:
class CatDogDataset(Dataset):
    def __init__(self, image_paths, transform):
        super().__init__()
        self.paths = image_paths
        self.len = len(self.paths)
        self.transform = transform
        
    def __len__(self): return self.len
    
    def __getitem__(self, index): 
        path = self.paths[index]
        image = Image.open(path).convert('RGB')
        image = self.transform(image)
        label = 0 if 'cat' in path else 1
        return (image, label)

In [8]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [9]:
import random

In [10]:
def train_path(p): return f"../data/train/{p}"
img_files = list(filter(lambda x: x != '../input/dogs-vs-cats-redux-kernels-edition/train/train', img_files))
img_files = list(map(train_path, img_files))

In [11]:
len(img_files)

25000

In [12]:
random.shuffle(img_files)
train_files = img_files[:20000]
valid = img_files[20000:]

In [13]:
train_files[:5]

['../data/train/cat.1484.jpg',
 '../data/train/dog.9818.jpg',
 '../data/train/dog.10066.jpg',
 '../data/train/cat.5039.jpg',
 '../data/train/dog.9905.jpg']

In [14]:
len(train_files)

20000

In [15]:
len(valid)

5000

In [16]:
train_ds = CatDogDataset(train_files, transform)
train_dl = DataLoader(train_ds, batch_size=100)
len(train_ds), len(train_dl)

(20000, 200)

In [17]:
valid_ds = CatDogDataset(valid, transform)
valid_dl = DataLoader(valid_ds, batch_size=100)
len(valid_ds), len(valid_dl)

(5000, 50)

# Transfer Learning

In [18]:
model = models.densenet121(pretrained=True)

Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /tmp/.cache/torch/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 55.0MB/s]


This model is built out of two main parts, the features and the classifier. The features part is a stack of convolutional layers and overall works as a feature detector that can be fed into a classifier. The classifier part is a single fully-connected layer `(classifier): Linear(in_features=1024, out_features=1000)`. This layer was trained on the ImageNet dataset, so it won't work for our specific problem. That means we need to replace the classifier, but the features will work perfectly on their own. In general, I think about pre-trained networks as amazingly good feature detectors that can be used as the input for simple feed-forward classifiers.

In [19]:
# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False

from collections import OrderedDict
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(1024, 512)),
                          ('relu', nn.ReLU()),
                          ('fc2', nn.Linear(512, 2)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
    
model.classifier = classifier
model.cuda()

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [20]:
losses = []
accuracies = []
epoches = 5
start = time.time()
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

In [21]:
for epoch in range(epoches):
    epoch_loss = 0
    epoch_accuracy = 0
    for X, y in train_dl:
        X = X.cuda()
        y = y.cuda()
        preds = model(X)
        preds = torch.exp(preds)
        loss = loss_fn(preds, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        accuracy = ((preds.argmax(dim=1) == y).float().mean())
        epoch_accuracy += accuracy
        epoch_loss += loss
        print('.', end='', flush=True)
        
    epoch_accuracy = epoch_accuracy/len(train_dl)
    accuracies.append(epoch_accuracy)
    epoch_loss = epoch_loss / len(train_dl)
    losses.append(epoch_loss)
    print("Epoch: {}, train loss: {:.4f}, train accuracy: {:.4f}, time: {}".format(epoch, epoch_loss, epoch_accuracy, time.time() - start))


    with torch.no_grad():
        val_epoch_loss = 0
        val_epoch_accuracy = 0
        for val_X, val_y in valid_dl:
            val_X = val_X.cuda()
            val_y = val_y.cuda()
            val_preds = model(val_X)
            val_preds = torch.exp(val_preds)
            val_loss = loss_fn(val_preds, val_y)

            val_epoch_loss += val_loss            
            val_accuracy = ((val_preds.argmax(dim=1) == val_y).float().mean())
            val_epoch_accuracy += val_accuracy
        val_epoch_accuracy = val_epoch_accuracy/len(valid_dl)
        val_epoch_loss = val_epoch_loss / len(valid_dl)
        print("Epoch: {}, valid loss: {:.4f}, valid accuracy: {:.4f}, time: {}\n".format(epoch, val_epoch_loss, val_epoch_accuracy, time.time() - start))

........................................................................................................................................................................................................Epoch: 0, train loss: 0.3423, train accuracy: 0.9713, time: 130.61530137062073
Epoch: 0, valid loss: 0.3478, valid accuracy: 0.9644, time: 161.64134049415588

........................................................................................................................................................................................................Epoch: 1, train loss: 0.3296, train accuracy: 0.9835, time: 287.3201825618744
Epoch: 1, valid loss: 0.3403, valid accuracy: 0.9720, time: 318.42048740386963

........................................................................................................................................................................................................Epoch: 2, train loss: 0.3296, train accuracy: 0.9834, time: 444.7429702281952
Epoch